<a href="https://colab.research.google.com/github/sheikhmt/COE379-ML-project/blob/main/extract_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw
!pip install google-cloud-storage
!pip install nltk


In [ ]:
import praw
import json
import os
import time
from datetime import datetime, timedelta
from google.cloud import storage
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive/')
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/My Drive/Colab Notebooks/reddit-data-access-key/coe379-ml-project-36d4c48ef4f4.json"


In [ ]:
from google.cloud import storage
storage_client = storage.Client()
print("Google Cloud Storage client initialized successfully.")


Google Cloud Storage client initialized successfully.


In [ ]:
import praw
import json
import os
from google.cloud import storage
from datetime import datetime, timedelta
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize NLTK's Sentiment Analyzer (download if not already done)
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Set up your Reddit API client
reddit = praw.Reddit(
    client_id="RHWB5hBX8j-hIRf_0EIpig",
    client_secret="sExW-IPfD-JZwyGIM4Q9maVAoyn1cw",
    user_agent="reddit-data-access"
)

# Define Google Cloud Storage client
storage_client = storage.Client()
bucket_name = "fin_analysis_data"


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
def fetch_and_upload_reddit_data(company_name):
    # Define the Google Cloud bucket path
    gcs_path = f"fin_text/{company_name}/social_media/reddit/{company_name}_reddit_data.json"

    # Calculate date range (last three months)
    current_time = datetime.utcnow()
    three_months_ago = current_time - timedelta(days=90)

    # Fetch Reddit posts for the given company
    posts_data = []
    for submission in reddit.subreddit("all").search(company_name, sort="top", time_filter="all", limit=100):
        # Convert submission timestamp to datetime
        submission_date = datetime.utcfromtimestamp(submission.created_utc)

        # Filter by date range
        if submission_date >= three_months_ago:
            # Perform sentiment analysis
            sentiment_score = sia.polarity_scores(submission.selftext)['compound']

            # Append relevant data
            posts_data.append({
                "title": submission.title,
                "text": submission.selftext,
                "upvotes": submission.score,
                "date": submission_date.isoformat(),
                "sentiment_score": sentiment_score
            })

    # Save data to a JSON file
    json_file_name = f"{company_name}_reddit_data.json"
    with open(json_file_name, "w") as json_file:
        json.dump(posts_data, json_file, indent=4)

    # Upload to Google Cloud Storage
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(gcs_path)
    blob.upload_from_filename(json_file_name)
    print(f"Uploaded {json_file_name} to {gcs_path} in bucket {bucket_name}")

    # Clean up local JSON file after upload
    os.remove(json_file_name)

# Run the data extraction and upload function for each company
fetch_and_upload_reddit_data("honeywell")
fetch_and_upload_reddit_data("lockheed")
fetch_and_upload_reddit_data("raytheon")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Uploaded honeywell_reddit_data.json to fin_text/honeywell/social_media/reddit/honeywell_reddit_data.json in bucket fin_analysis_data


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Uploaded lockheed_reddit_data.json to fin_text/lockheed/social_media/reddit/lockheed_reddit_data.json in bucket fin_analysis_data
Uploaded raytheon_reddit_data.json to fin_text/raytheon/social_media/reddit/raytheon_reddit_data.json in bucket fin_analysis_data
